In [1]:
import sqlite3
import os
import dotenv
from tqdm import tqdm

dotenv.load_dotenv()

conn = sqlite3.connect(os.getenv("DB_FILE"))
cur = conn.cursor()

In [ ]:
import re

institutii = cur.execute("SELECT id FROM institutii").fetchall()

icon_files = os.listdir('../data/icons')

print(icon_files)

for (id,) in tqdm(institutii):
    icons = [
        icon
        for icon in icon_files
        if re.match(r"{}(_\d+)?\.(png|ico|jpg|jpeg|gif|webp|svg)".format(id), icon)
    ]
    # print(id, icons)

    if len(icons) >= 1:
        conn.execute("UPDATE institutii SET sigla = ? WHERE id = ?", (icons[0], id))
        # print(id, icons[0])
conn.commit()

In [11]:
import re

institutii = cur.execute("SELECT id FROM institutii").fetchall()

icon_files = os.listdir('../../bacplus/public/icons-xs')

print(icon_files)

for (id,) in tqdm(institutii):
    icons = [
        icon
        for icon in icon_files
        if re.match(r"{}(_\d+)?\.(png|ico|jpg|jpeg|gif|webp|svg)".format(id), icon)
    ]
    # print(id, icons)

    if len(icons) >= 1:
        conn.execute("UPDATE institutii SET sigla_xs = ? WHERE id = ?", ("da", id))
        # print(id, icons[0])
conn.commit()

['CENTRUL_SCOLAR_DE_EDUCATIE_INCLUZIVA_ALBATROS_CONSTANTA_CT.webp', 'CENTRUL_SCOLAR_PENTRU_EDUCATIE_INCLUZIVA_ALBATROS_CONSTANTA_CT.webp', 'CENTRUL_SCOLAR_PENTRU_EDUCATIE_INCLUZIVA_ELENA_DOAMNA_FOCSANI_VN.webp', 'COLEGIUL_AGRICOL_DIMITRIE_CANTEMIR_MUN_HUSI_VS.webp', 'COLEGIUL_AGRICOL_SI_DE_INDUSTRIE_ALIMENTARA_VASILE_ADAMACHI_IASI_IS.webp', 'COLEGIUL_AGRICOL_TRAIAN_SAVULESCU_TIRGU_MURES_MS.webp', 'COLEGIUL_ANDRONIC_MOTRESCU_RADAUTI_SV.webp', 'COLEGIUL_COMERCIAL_CAROL_I_CONSTANTA_CT.webp', 'COLEGIUL_DE_ARTA_CARMEN_SYLVA_MUNICIPIUL_PLOIESTI_PH.webp', 'COLEGIUL_DE_ARTE_BAIA_MARE_MM.webp', 'COLEGIUL_DE_ARTE_SABIN_DRAGOI_ARAD_AR.webp', 'COLEGIUL_DE_MUZICA_SIGISMUND_TODUTA_CLUJ_NAPOCA_CJ.webp', 'COLEGIUL_DE_SERVICII_IN_TURISM_NAPOCA_CLUJ_NAPOCA_CJ.webp', 'COLEGIUL_DE_STIINTE_ALE_NATURII_EMIL_RACOVITA_BRASOV_BV.webp', 'COLEGIUL_DE_STIINTE_GRIGORE_ANTIPA_BRASOV_BV.webp', 'COLEGIUL_DOBROGEAN_SPIRU_HARET_TULCEA_TL.webp', 'COLEGIUL_ECOLOGIC_PROF_UNIV_DR_ALEXANDRU_IONESCU_PITESTI_AG.webp', 'COLEGI

100%|██████████| 6985/6985 [00:32<00:00, 215.93it/s] 


In [17]:

from judete import get_county_code, judete_dupa_cod

institutii = cur.execute("SELECT id, adresa, website, cod_siiir FROM institutii").fetchall()

for (id, adresa, website, cod_siiir) in tqdm(institutii):
  if cod_siiir is not None:
    siiir = cur.execute("SELECT strada, nr_strada, localitate, cod_postal, website, telefon, cod_fiscal FROM siiir WHERE cod = ?", (cod_siiir,)).fetchone()

    if not adresa:
      adresa_siiir = siiir[0].strip(' ,.')
      if siiir[1].strip() not in ["", "-", ".", "x"]:
        adresa_siiir += " " + siiir[1].strip(' ,.')
      if siiir[2] is not None:
        adresa_siiir += ", " + siiir[2].title()
      if siiir[3] is not None:
        adresa_siiir += ", " + siiir[3]

      cur.execute("UPDATE institutii SET adresa = ? WHERE id = ?", (adresa_siiir, id))
    
    if not website and siiir[4]:
      cur.execute("UPDATE institutii SET website = ? WHERE id = ?", (siiir[4], id))
    
    if siiir[5]:
      cur.execute("UPDATE institutii SET telefon = ? WHERE id = ?", (siiir[5], id))
    
    if siiir[6]:
      cur.execute("UPDATE institutii SET cod_fiscal = ? WHERE id = ?", (siiir[6], id))

conn.commit()

  0%|          | 0/6999 [00:00<?, ?it/s]

100%|██████████| 6999/6999 [00:00<00:00, 7227.18it/s]


In [20]:
conn.close()
conn = sqlite3.connect(os.getenv("DB_FILE"))
cur = conn.cursor()

scoli = cur.execute(
    "SELECT id_scoala FROM scoli"
).fetchall()

for id_scoala in (scoli):
    cur.execute(
        "UPDATE institutii SET gimnaziu = 'da' WHERE id = ?",
        (id_scoala),
    )

print(f"Updated {len(scoli)} scoli.")

liceu = cur.execute(
    "SELECT id_liceu FROM licee"
).fetchall()

for id_liceu in (liceu):
    cur.execute(
        "UPDATE institutii SET liceu = 'da' WHERE id = ?",
        (id_liceu),
    )

print(f"Updated {len(liceu)} licee.")

conn.commit()

Updated 6206 scoli.
Updated 1664 licee.
